In [1]:

from os.path import dirname, join
from os import listdir
from pprint import pprint

import pydicom

from pydicom.filereader import read_dicomdir


import matplotlib.pyplot as plt
import numpy as np
import scipy

from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

from skimage.segmentation import clear_border
from scipy import ndimage as ndi
from skimage.filters import roberts, sobel

import cv2
import glob

In [3]:
def rescale_patient_images(images_zyx, org_spacing_xyz, target_voxel_mm, is_mask_image=True, verbose=False):
    if verbose:
        print("Spacing: ", org_spacing_xyz)
        print("Shape: ", images_zyx.shape)

    # print "Resizing dim z"
    resize_x = 1.0
    resize_y = float(org_spacing_xyz[2]) / float(target_voxel_mm)
    interpolation = cv2.INTER_NEAREST if is_mask_image else cv2.INTER_LINEAR
    res = cv2.resize(images_zyx, dsize=None, fx=resize_x, fy=resize_y, interpolation=interpolation)  # opencv assumes y, x, channels umpy array, so y = z pfff
    # print "Shape is now : ", res.shape

    res = res.swapaxes(0, 2)
    res = res.swapaxes(0, 1)
    # print "Shape: ", res.shape
    resize_x = float(org_spacing_xyz[0]) / float(target_voxel_mm)
    resize_y = float(org_spacing_xyz[1]) / float(target_voxel_mm)

    # cv2 can handle max 512 channels..
    if res.shape[2] > 512:
        res = res.swapaxes(0, 2)
        res1 = res[:256]
        res2 = res[256:]
        res1 = res1.swapaxes(0, 2)
        res2 = res2.swapaxes(0, 2)
        res1 = cv2.resize(res1, dsize=None, fx=resize_x, fy=resize_y, interpolation=interpolation)
        res2 = cv2.resize(res2, dsize=None, fx=resize_x, fy=resize_y, interpolation=interpolation)
        res1 = res1.swapaxes(0, 2)
        res2 = res2.swapaxes(0, 2)
        res = numpy.vstack([res1, res2])
        res = res.swapaxes(0, 2)
    else:
        res = cv2.resize(res, dsize=None, fx=resize_x, fy=resize_y, interpolation=interpolation)

    # channels = cv2.split(res)
    # resized_channels = []
    # for channel in  channels:
    #     channel = cv2.resize(channel, dsize=None, fx=resize_x, fy=resize_y)
    #     resized_channels.append(channel)
    # res = cv2.merge(resized_channels)
    # print "Shape after resize: ", res.shape
    res = res.swapaxes(0, 2)
    res = res.swapaxes(2, 1)
    if verbose:
        print("Shape after: ", res.shape)
    return res

In [4]:
filelist = glob.glob("*.png")

imlist = [plt.imread(file) for file in filelist]
    
R_slices = np.stack([im[:,:,0] for im in imlist])
G_slices = np.stack([im[:,:,1] for im in imlist])
B_slices = np.stack([im[:,:,2] for im in imlist])

In [5]:
R_slices = np.array(R_slices, dtype=np.int16)
print(R_slices.shape)

(166, 1102, 1102)


In [ ]:
def resample(image, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    pixel_spacing = ['2.5','0.310546875', '0.310546875']
    spacing = map(float, (pixel_spacing))

    spacing = np.array(list(spacing))

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor)
    
    return image, new_spacing

print("Shape before resampling\t", R_slices.shape)
imgs_after_resamp, spacing = resample(R_slices, [1,1,1])
print("Shape after resampling\t", imgs_after_resamp.shape)

Shape before resampling	 (166, 1102, 1102)


In [21]:
def make_mesh(image, threshold=-300, step_size=1):

    p = image.transpose(2,1,0)
    
    verts, faces, norm, val = measure.marching_cubes(p, threshold, step_size=step_size, allow_degenerate=True) 
    return verts, faces


def plt_3d(verts, faces):

    x,y,z = zip(*verts) 
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], linewidths=0.05, alpha=1)
    face_color = [1, 1, 0.9]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)

    ax.set_xlim(0, max(x))
    ax.set_ylim(0, max(y))
    ax.set_zlim(0, max(z))
    ax.set_axis_bgcolor((0.7, 0.7, 0.7))
    plt.show()

In [22]:
pixel_spacing = ['0.310546875', '0.310546875', '3.0']
TARGET_VOXEL_MM = 1.00


#R_slices = rescale_patient_images(R_slices, pixel_spacing, TARGET_VOXEL_MM)
verts, faces = make_mesh(R_slices)
plt_3d(verts, faces)

ValueError: Surface level must be within volume data range.